In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# %pip install pandas_summary

In [3]:
# # split train y val
# n_split = int(df_full.shape[0]*0.2)
# df_train = df_full[n_split:]
# df_val = df_full[:n_split]
# len(df_train), len(df_val)

In [4]:
df_full = pd.read_feather('../data/full_data.fth')
df_full = df_full.sort_values(by='Date_rating')
df_full.head(2)

,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,rating,Date_rating,Name_movie,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
25416,259,Allen Oliver,1977,M,student,1997-09-20 03:05:10,0,0,9,255,4,1997-09-20 03:05:10,My Best Friend's Wedding (1997),1997-06-20,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,27564
25457,259,Allen Oliver,1977,M,student,1997-09-20 03:05:10,0,0,9,286,4,1997-09-20 03:05:27,"English Patient, The (1996)",1996-11-15,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,27347


In [5]:
df_full.columns

Index(['user_id', 'Full Name', 'year of birth', 'Gender', 'Occupation',
       'Active Since', 'Trabajador', 'Antiguedad', 'Horoscopo Activacion',
       'movie_id', 'rating', 'Date_rating', 'Name_movie', 'Release Date',
       'Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'Antiguedad_pelicula'],
      dtype='object')

## Separo en categoricas y numericas, los genres los dejare como binarios

In [6]:
cat_vars = ['Gender', 'Occupation', 'Horoscopo Activacion']
df_full[cat_vars].head()

,Gender,Occupation,Horoscopo Activacion
25416,M,student,9
25457,M,student,9
25445,M,student,9
25435,M,student,9
25455,M,student,9


In [7]:
contin_vars = [ 'year of birth','Trabajador','Antiguedad', 'Antiguedad_pelicula']

df_full[contin_vars].head()

,year of birth,Trabajador,Antiguedad,Antiguedad_pelicula
25416,1977,0,0,27564
25457,1977,0,0,27347
25445,1977,0,0,27571
25435,1977,0,0,13879
25455,1977,0,0,23741


In [8]:
DataFrameSummary(df_full[contin_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
year of birth,61,numeric,0
Trabajador,2,bool,0
Antiguedad,196,numeric,0
Antiguedad_pelicula,239,numeric,0


# Normalizacion y labelencode

In [9]:
from sklearn_pandas import DataFrameMapper

In [10]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [11]:
mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_full)

In [12]:
N = 100
print(df_full['Occupation'].values[30:35])
print(mapper_cat.transform(df_full)[:, 2][30:35])

['student' 'student' 'student' 'student' 'student']
[4 4 4 4 4]


In [15]:
N = 100
print(df_full['user_id'].values[30:35])
print(mapper_cat.transform(df_full)[:, 2][30:35])

[712 712 712 712 712]
[4 4 4 4 4]


In [16]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_full)

In [17]:
N = 5
print(df_full['Antiguedad_pelicula'].values[:N])
print(mapper_cont.transform(df_full)[:, 0][:N])

[27564 27347 27571 13879 23741]
[1.03524547 1.03524547 1.03524547 1.03524547 1.03524547]


In [18]:
# La hacemos con todas ahora
df_full[cat_vars] = mapper_cat.transform(df_full)
df_full[contin_vars] = mapper_cont.transform(df_full)

In [19]:
df_full = df_full.reset_index()
df_full.head()

,index,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,rating,Date_rating,Name_movie,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
0,25416,259,Allen Oliver,1.035245,1,19,1997-09-20 03:05:10,-0.112969,-1.453873,4,255,4,1997-09-20 03:05:10,My Best Friend's Wedding (1997),1997-06-20,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.657124
1,25457,259,Allen Oliver,1.035245,1,19,1997-09-20 03:05:10,-0.112969,-1.453873,4,286,4,1997-09-20 03:05:27,"English Patient, The (1996)",1996-11-15,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0.615416
2,25445,259,Allen Oliver,1.035245,1,19,1997-09-20 03:05:10,-0.112969,-1.453873,4,298,4,1997-09-20 03:05:54,Face/Off (1997),1997-06-27,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.658469
3,25435,259,Allen Oliver,1.035245,1,19,1997-09-20 03:05:10,-0.112969,-1.453873,4,185,4,1997-09-20 03:06:21,Psycho (1960),1960-01-01,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,-1.973172
4,25455,259,Allen Oliver,1.035245,1,19,1997-09-20 03:05:10,-0.112969,-1.453873,4,173,4,1997-09-20 03:07:23,"Princess Bride, The (1987)",1987-01-01,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-0.077668


In [20]:
df_full.to_feather('../data/full_normalized_data.fth')